### Imports 

In [1]:
# Data Management
import numpy as np
import pandas as pd
import polars as pl  # Using Polars for data management

# Data Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Deep Learning
import torch
from torch.utils.data import Dataset, DataLoader

# Visualization
import matplotlib.pyplot as plt

### Data Extraction

In [2]:
# Load data from saved file using Polars
df = pl.read_csv("data/HMM-SPY.csv")

print(f"Length: {len(df)}")

df.columns

Length: 852


['Date',
 'Open',
 'Adj Close',
 'Returns',
 'Range',
 'MA_12',
 'MA_21',
 'HMM',
 'MA_Signal',
 'HMM_Signal',
 'Main_Signal',
 'lrets_bench',
 'bench_prod',
 'bench_prod_exp',
 'lrets_strat',
 'lrets_prod',
 'strat_prod_exp']

In [3]:
df_uf = df.clone()
useful_features = ["Open", "Adj Close", "MA_12", "MA_21", "HMM", "lrets_bench", "lrets_strat"]
df_uf = df.select(useful_features)

df_uf.head(2)

Open,Adj Close,MA_12,MA_21,HMM,lrets_bench,lrets_strat
f64,f64,f64,f64,i64,f64,f64
256.820007,242.321487,233.62229,238.259514,2,0.009351,0.0
257.559998,243.454041,234.584504,238.076622,3,0.004663,-0.0


In [5]:
df_uf.dtypes

[Float64, Float64, Float64, Float64, Int64, Float64, Float64]